In [1]:
from glob import glob
import os
import pandas as pd
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [7]:
for csv_path in glob("data/MSD/*"):
    df = pd.read_csv(csv_path, index_col=0)
    df["gt_path"] = df["gt_path"].str.replace("/data/datasets/MSD/", "")
    df.to_csv(csv_path)

In [ ]:
root = "/hpcfs/users/a1232079/duyanh/MedSAM2/datasets/MSD"
for task in os.listdir(root):
    if task.startswith("."):
        continue
    
    for subset in ["labelsTr", "labelsTs"]:
        sample = []
        for gt_path in tqdm(glob(os.path.join(root, task, subset, "*")), desc=f"{task}_{subset}"):
        sample = []
        for gt_path in tqdm(glob(os.path.join(root, task, subset, "*")), desc=f"{task}_{subset}"):
            gt_raw = nib.load(gt_path)
            gt = np.asanyarray(gt_raw.dataobj)
            sample.extend([(task, gt_path, label) for label in np.unique(gt) if label != 0])
        df = pd.DataFrame(sample, columns=["task", "gt_path", "obj_id"])
        df.to_csv(f"./data/MSD/{task}_{subset}.csv")